<a href="https://colab.research.google.com/github/HarshAg1702/Speech_emotion_recognition/blob/main/emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import zipfile
import requests

url = "https://www.kaggle.com/api/v1/datasets/download/uwrfkaggler/ravdess-emotional-speech-audio?datasetVersionNumber=1"

response = requests.get(url)
zip_path = "/content/ravdess.zip"

# Write the content to a file
with open(zip_path, "wb") as file:
    file.write(response.content)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/ravdess")

# Path to the extracted dataset
RAVDESS = "/content/ravdess"

os.remove(zip_path)

path_to_delete = "/content/ravdess/audio_speech_actors_01-24"

if os.path.exists(path_to_delete):
    shutil.rmtree(path_to_delete)
else:
    print(f"Path does not exist: {path_to_delete}")

In [ ]:
# Paths to the directories
train_set_path = os.path.join(RAVDESS, "Train Set")
test_set_path = os.path.join(RAVDESS, "Test Set")

# Create the subfolders if they don't exist
os.makedirs(train_set_path, exist_ok=True)
os.makedirs(test_set_path, exist_ok=True)

# List of actor folders in the RAVDESS directory
actor_folders = sorted([f for f in os.listdir(RAVDESS) if f.startswith("Actor_")])

for actor in actor_folders[0:20]:
    shutil.move(os.path.join(RAVDESS, actor), train_set_path)

for actor in actor_folders[20:24]:
    shutil.move(os.path.join(RAVDESS, actor), test_set_path)

# **Filename identifiers as per the official RAVDESS website:**

Modality (01 = full-AV, 02 = video-only, 03 = audio-only).

Vocal channel (01 = speech, 02 = song).

Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 =
fearful, 07 = disgust, 08 = surprised).

Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.

Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").

Repetition (01 = 1st repetition, 02 = 2nd repetition).
Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

## Example:

02-01-06-01-02-01-12.mp4

This means the meta data for the audio file is:

Video-only (02)

Speech (01)

Fearful (06)

Normal intensity (01)

Statement "dogs" (02)

1st Repetition (01)

12th Actor (12) - Female (as the actor ID number is even)

In [ ]:
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions = ['happy', 'sad', 'angry', 'fearful', 'neutral']

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import librosa
import librosa.display

from glob import glob
from IPython.display import Audio

In [ ]:
train_set_list = os.listdir(train_set_path)

data = []

for dir in train_set_list:
  actor = os.listdir(train_set_path+ '/'+ dir)
  for file in actor:
    part = file.split('.')[0]
    part = part.split('-')
    emotion = emotions.get(part[2], None)
    if emotion is not None and emotion in observed_emotions:
      file_path = os.path.join(train_set_path, dir, file)
      data.append({'Emotion': emotion, 'File_Path': file_path})

ravdess_df = pd.DataFrame(data)
print(ravdess_df)

     Emotion                                          File_Path
0        sad  /content/ravdess/Train Set/Actor_08/03-01-04-0...
1    fearful  /content/ravdess/Train Set/Actor_08/03-01-06-0...
2    fearful  /content/ravdess/Train Set/Actor_08/03-01-06-0...
3    neutral  /content/ravdess/Train Set/Actor_08/03-01-01-0...
4      angry  /content/ravdess/Train Set/Actor_08/03-01-05-0...
..       ...                                                ...
715    angry  /content/ravdess/Train Set/Actor_11/03-01-05-0...
716    happy  /content/ravdess/Train Set/Actor_11/03-01-03-0...
717  fearful  /content/ravdess/Train Set/Actor_11/03-01-06-0...
718  neutral  /content/ravdess/Train Set/Actor_11/03-01-01-0...
719    angry  /content/ravdess/Train Set/Actor_11/03-01-05-0...

[720 rows x 2 columns]


In [ ]:
import soundfile as sf

# Constants
NOISE_FACTOR = 0.035
SHIFT_MAX = 0.5
PITCH_STEPS = 2
SPEED_FACTOR = 1.5

# Function to add noise to a signal
def add_noise(data, noise_factor=NOISE_FACTOR):
    noise = np.random.randn(len(data))
    augmented_data = data + noise_factor * noise
    return augmented_data

# Function to shift a signal in time
def shift_time(data, shift_max=SHIFT_MAX):
    shift = np.random.randint(int(len(data) * shift_max))
    augmented_data = np.roll(data, shift)
    return augmented_data

# Function to change the pitch of a signal
def change_pitch(data, sampling_rate, pitch_steps=PITCH_STEPS):
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=pitch_steps)

# Function to change the speed of a signal
def change_speed(data, speed_factor=SPEED_FACTOR):
    return librosa.effects.time_stretch(data, rate=speed_factor)

# Apply data augmentation
augmented_data = []

for index, row in ravdess_df.iterrows():
    y, sr = librosa.load(row['File_Path'])

    # Apply noise injection
    noisy_data = add_noise(y)
    noisy_file_path = row['File_Path'].replace(".wav", "_noisy.wav")
    sf.write(noisy_file_path, noisy_data, sr)
    augmented_data.append({'Emotion': row['Emotion'], 'File_Path': noisy_file_path})

    # Apply time shifting
    shifted_data = shift_time(y)
    shifted_file_path = row['File_Path'].replace(".wav", "_shifted.wav")
    sf.write(shifted_file_path, shifted_data, sr)
    augmented_data.append({'Emotion': row['Emotion'], 'File_Path': shifted_file_path})

    # Apply pitch shifting
    pitched_data = change_pitch(y, sr)
    pitched_file_path = row['File_Path'].replace(".wav", "_pitched.wav")
    sf.write(pitched_file_path, pitched_data, sr)
    augmented_data.append({'Emotion': row['Emotion'], 'File_Path': pitched_file_path})

    # Apply speed change
    try:
        speed_data = change_speed(y, SPEED_FACTOR)
        speed_file_path = row['File_Path'].replace(".wav", "_speed.wav")
        sf.write(speed_file_path, speed_data, sr)
        augmented_data.append({'Emotion': row['Emotion'], 'File_Path': speed_file_path})
    except Exception as e:
        print(f"Error processing file {row['File_Path']} with speed factor {SPEED_FACTOR}: {e}")

augmented_df = pd.DataFrame(augmented_data)
final_df = pd.concat([ravdess_df, augmented_df], ignore_index=True)

print(final_df)

      Emotion                                          File_Path
0         sad  /content/ravdess/Train Set/Actor_08/03-01-04-0...
1     fearful  /content/ravdess/Train Set/Actor_08/03-01-06-0...
2     fearful  /content/ravdess/Train Set/Actor_08/03-01-06-0...
3     neutral  /content/ravdess/Train Set/Actor_08/03-01-01-0...
4       angry  /content/ravdess/Train Set/Actor_08/03-01-05-0...
...       ...                                                ...
3595  neutral  /content/ravdess/Train Set/Actor_11/03-01-01-0...
3596    angry  /content/ravdess/Train Set/Actor_11/03-01-05-0...
3597    angry  /content/ravdess/Train Set/Actor_11/03-01-05-0...
3598    angry  /content/ravdess/Train Set/Actor_11/03-01-05-0...
3599    angry  /content/ravdess/Train Set/Actor_11/03-01-05-0...

[3600 rows x 2 columns]


In [ ]:
# Function to extract features from an audio file
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs, axis=1)

    # Extract Chroma feature
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)

    # Extract Spectral Contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_contrast_mean = np.mean(spectral_contrast, axis=1)

    # Extract Zero Crossing Rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    zero_crossing_rate_mean = np.mean(zero_crossing_rate)

    # Extract Root Mean Square Energy
    S = np.abs(librosa.stft(y))
    rmse = librosa.feature.rms(S=S)
    rmse_mean = np.mean(rmse)

    # Extract Mel-spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mel_spectrogram_mean = np.mean(mel_spectrogram, axis=1)

    # Concatenate all features
    features = np.hstack((mfccs_mean, chroma_mean, spectral_contrast_mean, zero_crossing_rate_mean, rmse_mean, mel_spectrogram_mean))

    return features

# Extract features for each file in the final_df
features_list = []
for index, row in final_df.iterrows():
    features = extract_features(row['File_Path'])
    features_list.append(features)

# Convert features_list to a DataFrame
features_df = pd.DataFrame(features_list)

# Add the Emotion column from final_df to features_df
features_df['Emotion'] = final_df['Emotion']

print(features_df)

               0          1          2          3          4          5  \
0    -761.483032  71.773376  14.633630  23.550846   8.441536  18.252174   
1    -578.749084  77.713196  -7.886116   6.420325   6.179074  11.582044   
2    -603.779297  64.604401  -6.578300   7.780734   0.403219  11.610985   
3    -712.075562  76.019432  12.366280  21.779139   8.561011  15.253534   
4    -563.111694  60.146988   0.494722  13.174312   3.693977   1.640879   
...          ...        ...        ...        ...        ...        ...   
3595 -745.760376  59.496197  24.346766  18.837852   9.742865   5.289154   
3596 -100.514687   9.567369  -0.561821  -1.624645  -5.292121  -3.825037   
3597 -350.797241  23.505442  -7.981641   1.008470 -12.795454  -5.142152   
3598 -386.030731  22.522659 -11.340427  -2.031952 -15.844463  -2.403362   
3599 -378.480286  27.758413  -9.050571  -0.054251 -14.226600  -5.936737   

             6          7          8          9  ...           153  \
0     0.082437  11.601342  -4

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the features data and transform it
features_scaled = scaler.fit_transform(features_df.drop('Emotion', axis=1))

# Convert the scaled features back to a DataFrame
features_scaled_df = pd.DataFrame(features_scaled, columns=features_df.columns[:-1])

# Add the Emotion column back to the scaled features DataFrame
features_scaled_df['Emotion'] = features_df['Emotion']

print(features_scaled_df.head())

          0         1         2         3         4         5         6  \
0 -1.274558  1.185898  1.578318  2.114246  1.179709  2.369706  0.956625   
1 -0.408729  1.413925 -0.113551  0.011961  0.895878  1.588487 -0.085357   
2 -0.527327  0.910685 -0.015297  0.178912  0.171282  1.591877 -0.556579   
3 -1.040456  1.348902  1.407976  1.896819  1.194698  2.018499 -0.091963   
4 -0.334636  0.739567  0.516087  0.840821  0.584116  0.424159 -0.313337   

          7         8         9  ...       153       154       155       156  \
0  2.555333  0.249841 -0.399751  ... -0.516611 -0.516778 -0.517168 -0.517737   
1  2.534573  0.113431 -1.187358  ... -0.516469 -0.516679 -0.517110 -0.517695   
2  1.954285 -0.098310 -1.974330  ... -0.516443 -0.516631 -0.517064 -0.517652   
3  2.751052 -0.509037 -0.004057  ... -0.516612 -0.516781 -0.517171 -0.517740   
4  2.339122  1.026198 -0.203198  ... -0.516563 -0.516745 -0.517135 -0.517707   

        157       158       159       160       161  Emotion  
0 -0.

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers

# # Define the neural network model
# model = tf.keras.Sequential([
#     layers.Dense(64, activation='relu', input_shape=(features_df.shape[1]-1,)),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(5, activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # One-hot encode the Emotion column
# emotion_labels = pd.get_dummies(features_scaled_df['Emotion']).to_numpy()

# # Train the model using the entire dataset
# model.fit(features_df.drop('Emotion', axis=1), emotion_labels, epochs=300)

# # Save the model parameters
# model.save_weights('/content/ravdess/softmax_regression_params.h5')
#######
import tensorflow as tf
from tensorflow.keras import layers

# Define the neural network model
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(features_df.shape[1]-1,)),
    layers.BatchNormalization(),  # Adding batch normalization
    layers.Dropout(0.5),  # Adding dropout for regularization
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dense(5, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the entire dataset
model.fit(features_df.drop('Emotion', axis=1), emotion_labels, epochs=300)

# Save the model parameters
model.save_weights('/content/ravdess/softmax_regression_params.h5')

















Epoch 1/300
113/113 [==============================] - 2s 4ms/step - loss: 1.7764 - accuracy: 0.2633
Epoch 2/300
113/113 [==============================] - 0s 4ms/step - loss: 1.5492 - accuracy: 0.3297
Epoch 3/300
113/113 [==============================] - 0s 4ms/step - loss: 1.4738 - accuracy: 0.3472
Epoch 4/300
113/113 [==============================] - 0s 4ms/step - loss: 1.4348 - accuracy: 0.3608
Epoch 5/300
113/113 [==============================] - 0s 4ms/step - loss: 1.3985 - accuracy: 0.3731
Epoch 6/300
113/113 [==============================] - 0s 4ms/step - loss: 1.3598 - accuracy: 0.3925
Epoch 7/300
113/113 [==============================] - 1s 5ms/step - loss: 1.3611 - accuracy: 0.3933
Epoch 8/300
113/113 [==============================] - 1s 5ms/step - loss: 1.3473 - accuracy: 0.4083
Epoch 9/300
113/113 [==============================] - 1s 6ms/step - loss: 1.3313 - accuracy: 0.4047
Epoch 10/300
113/113 [==============================] - 1s 5ms/step - loss: 1.3159 - accura

## **TESTING**

In [ ]:
test_set_list = os.listdir(test_set_path)

test_data = []

for dir in test_set_list:
  actor = os.listdir(test_set_path+ '/'+ dir)
  for file in actor:
    part = file.split('.')[0]
    part = part.split('-')
    emotion = emotions.get(part[2], None)
    if emotion is not None and emotion in observed_emotions:
      file_path = os.path.join(test_set_path, dir, file)
      test_data.append({'Emotion': emotion, 'File_Path': file_path})

test_ravdess_df = pd.DataFrame(test_data)

In [ ]:
test_features_list = []
for index, row in test_ravdess_df.iterrows():
    features = extract_features(row['File_Path'])
    test_features_list.append(features)

# Convert features_list to a DataFrame
test_features_df = pd.DataFrame(test_features_list)

# Add the Emotion column from final_df to features_df
test_features_df['Emotion'] = test_ravdess_df['Emotion']

In [ ]:
test_emotion_labels = pd.get_dummies(test_features_df['Emotion']).to_numpy()
test_features_df_modified = test_features_df.drop('Emotion', axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the features data and transform it
test_features_scaled = scaler.fit_transform(test_features_df.drop('Emotion', axis=1))

# Convert the scaled features back to a DataFrame
test_features_scaled_df = pd.DataFrame(test_features_scaled, columns=test_features_df.columns[:-1])

# Add the Emotion column back to the scaled features DataFrame
test_features_scaled_df['Emotion'] = test_features_df['Emotion']

print(test_features_scaled_df.head())

          0         1         2         3         4         5         6  \
0 -1.424114 -0.199530  0.512823 -0.368760 -0.433681 -1.199255  0.169433   
1  0.363747 -1.312970 -0.616007  0.305981 -1.258350 -1.271668 -0.142840   
2 -0.131468  0.131952  0.225756  0.862263 -0.624877 -0.774208 -0.632564   
3  0.333710 -0.299634 -1.493054 -0.480367 -0.201335 -1.220007 -2.495999   
4  0.852609 -1.159578 -1.422847  0.200169 -1.312053 -2.189700 -1.010546   

          7         8         9  ...       153       154       155       156  \
0 -0.369657 -1.054612 -0.131097  ... -0.605555 -0.600779 -0.600837 -0.577673   
1  0.061911 -1.096982  0.349940  ...  3.162275  2.229686  2.297152  2.986351   
2 -1.232867 -1.904443 -2.176419  ...  0.256370  0.594445  0.932813  0.965304   
3  0.200761 -0.252066 -0.225311  ... -0.331917 -0.306999 -0.283264 -0.330740   
4  0.319586 -0.753594 -0.552446  ...  1.199173  1.083483  1.216885  1.536872   

        157       158       159       160       161  Emotion  
0 -0.

In [ ]:
from sklearn.metrics import accuracy_score

def test_model(test_features_df_modified, test_emotion_labels):
    # Load the saved model parameters
    model.load_weights('/content/ravdess/softmax_regression_params.h5')

    # Make predictions on the test set
    y_pred = model.predict(test_features_df_modified)

    # Convert predictions to class labels
    y_pred_class = np.argmax(y_pred, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(np.argmax(test_emotion_labels, axis=1), y_pred_class)

    # Print accuracy in percentage
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Call the function
test_model(test_features_df_modified, test_emotion_labels)


5/5 [==============================] - 0s 4ms/step
Accuracy: 54.86%


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO)

def load_model_weights(model, filepath: str):
    try:
        model.load_weights(filepath)
        logging.info("Model weights loaded successfully.")
    except Exception as e:
        logging.error(f"Failed to load model weights: {e}")
        raise

def predict_labels(model, features: np.ndarray) -> np.ndarray:
    try:
        predictions = model.predict(features)
        predicted_classes = np.argmax(predictions, axis=1)
        logging.info("Predictions made successfully.")
        return predicted_classes
    except Exception as e:
        logging.error(f"Prediction failed: {e}")
        raise

def calculate_metrics(true_labels: np.ndarray, predicted_labels: np.ndarray):
    try:
        accuracy = accuracy_score(true_labels, predicted_labels)
        precision = precision_score(true_labels, predicted_labels, average='weighted')
        recall = recall_score(true_labels, predicted_labels, average='weighted')
        f1 = f1_score(true_labels, predicted_labels, average='weighted')

        # Logging the results
        logging.info(f"Accuracy: {accuracy * 100:.2f}%")
        logging.info(f"Precision: {precision * 100:.2f}%")
        logging.info(f"Recall: {recall * 100:.2f}%")
        logging.info(f"F1 Score: {f1 * 100:.2f}%")

        # Printing the results
        print(f"Accuracy: {accuracy * 100:.2f}%")
        print(f"Precision: {precision * 100:.2f}%")
        print(f"Recall: {recall * 100:.2f}%")
        print(f"F1 Score: {f1 * 100:.2f}%")

        return accuracy, precision, recall, f1
    except Exception as e:
        logging.error(f"Failed to calculate metrics: {e}")
        raise

def test_model(model, test_features_df_modified: np.ndarray, test_emotion_labels: np.ndarray, weights_filepath: str):
    """
    Test the model with the provided features and labels.

    Parameters:
    model: The machine learning model to be tested.
    test_features_df_modified (np.ndarray): Modified test features.
    test_emotion_labels (np.ndarray): True emotion labels for the test data.
    weights_filepath (str): Path to the saved model weights.

    Returns:
    tuple: Accuracy, precision, recall, and F1 score of the model on the test data.
    """
    # Load the saved model parameters
    load_model_weights(model, weights_filepath)

    # Make predictions on the test set
    y_pred_class = predict_labels(model, test_features_df_modified)

    # Calculate and return metrics
    return calculate_metrics(np.argmax(test_emotion_labels, axis=1), y_pred_class)

# Example call to the function
# Assuming 'model', 'test_features_df_modified', 'test_emotion_labels', and 'weights_filepath' are defined
accuracy, precision, recall, f1 = test_model(model, test_features_df_modified, test_emotion_labels, '/content/ravdess/softmax_regression_params.h5')

5/5 [==============================] - 0s 5ms/step
Accuracy: 54.86%
Precision: 53.46%
Recall: 54.86%
F1 Score: 53.34%
